# Working with a shared repo on a server 

First we are going to make a new directory and `clone` a copy of the repo into it. We get the path to use in the clone command from the repo, and the default folder will be use the repo name (we can specify a folder name if need be) 

In [ ]:
cd ~
$null = md repos
cd repos
git clone https://CAS-DFRP@dev.azure.com/CAS-DFRP/DFRP/_git/DFRP
cd .\DFRP

Cloning into 'DFRP'...

Our local repository knows it is linked to a remote repository. `git remote` will show us the information and in verbose mode will show exactly what we use for fetching from the server and pushing to the server (the URIs might be different)

In [1]:
cd ~\repos\DFRP
git remote -v

The below script needs to be able to find the current output cell; this is an easy method to get it.

origin	https://CAS-DFRP@dev.azure.com/CAS-DFRP/DFRP/_git/DFRP (fetch)
origin	https://CAS-DFRP@dev.azure.com/CAS-DFRP/DFRP/_git/DFRP (push)


Our remote has the default alias "origin" and we use the same URI for sending and receiving changes. 
In this example there is only one version of the repo on the server, but repos can be *forks* of other repos; for example on Github I contribute to another project. In that case `git remote -v` will know about another repository named "upstream"
```
git remote -v
origin     https://github.com/jhoneill/vsteam.git (fetch)
origin     https://github.com/jhoneill/vsteam.git (push)
upstream   https://github.com/DarqueWarrior/vsteam.git (fetch)
upstream   https://github.com/DarqueWarrior/vsteam.git (push)
```
Although git knows the URL for pushing to upstream, trying to do that will give an "access denied" error and I need to ask the the someone to *pull* changes from my repo into the upstream one, this is known as a *pull-request*

------


`Git fetch` will get branches and/or tags along with the objects necessary to complete their histories. (This shouldn't be necessary immediately after a clone, but will pick up new changes)    
`Git branch` shows us the branch(es) we are working with and `git branch -a` shows *all* the branches including the ones which are on the server but not set up locally.

In [ ]:
git fetch origin
git branch -a  

* master
  remotes/origin/HEAD -> origin/master
  remotes/origin/P10498824
  remotes/origin/Testbranch
  remotes/origin/develop
  remotes/origin/master
  remotes/origin/release

Again, when the origin is a fork of another repo, we may see additional branches. In my Github example, I have some local branches, and *origin* has set up some of the branches from the *upstream* repo. 
```
git branch -a  
  EnhanceGet-VsteamWorkItemType#301
* NewWorkitemtypeCommands
  master
  remotes/origin/EnhanceGet-VsteamWorkItemType#301
  remotes/origin/HEAD -> origin/master
  remotes/origin/NewWorkitemtypeCommands
  remotes/origin/hooks
  remotes/origin/master
  remotes/upstream/Bug345
  remotes/upstream/hooks
  remotes/upstream/master
````

------

We can start working on a branch which is linked to a different remote branch by doing a checkout with `-b` to specify *create a branch* and giving the *remote* branch as the start point.

In [ ]:
git status
''
git checkout -b develop origin/develop
''
git status


On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean

Branch 'develop' set up to track remote branch 'develop' from 'origin'.
Switched to a new branch 'develop'

On branch develop
Your branch is up to date with 'origin/develop'.

nothing to commit, working tree clean

Note that there are some states where we *can't* change branches. Git has the concept of a *stash* where we can put changes which aren't committed and would otherwise be lost.

We can see the sequence of earlier commits which have brought the branch to this state.

In [ ]:
git log --oneline --decorate | select -First 5
#For the notebook only get the first few lines.

5495636 (HEAD -> develop, tag: 19_develop) Update version variables and graphs ***NO_DEPLOY***
8683936 (tag: 17_develop) Sync Release 0.1 from Master
5942680 (tag: MasterValidation_runID_18, origin/master, origin/HEAD, master) Merged PR 2: Release 0.1
5a996b2 Add Documents ***NO_CI***
5e48807 (tag: 15) fix ps warning message

`--Decorate` shows some extra log information. In the first line
* `5495636` is the start of the hash for that commit. 
* `HEAD -> develop` means the current working area points here and it is on the "develop" branch 
* `tag: 19_develop` means we have an extra label for the commit. *Branches* are labels we can check out and work with and are usually the current version of something. Tags can be *any* label we want, but are usually historical notes. 
* `Update version variables and graphs ***NO_DEPLOY***` is a commit message 

On the third line, we can see one of the `5942680` was the version-before-last of the develop branch, but it is the current version of the local branch "master", the origin branch "master" and pointed to by HEAD at the origin (i.e. "master" is the current default branch at the origin)

In this case we have a Continuous Integration pipeline which runs when a commit occurs. It puts a tag link to a build (here the build ID is *SequenceNumber_BranchName*. `***NO_DEPLOY***` is a message to the CI system, in this case saying "Build but do not deploy" .  The fourth line has a message `***NO_CI***` which tells the CI pipeline not to run at all. 

We can now make changes in our local copy of this branch of the repo.

In [ ]:
## Add a file e.g. copy '~Git tutorial.ipynb' .\Git_Repos.ipynb
Git status
''
Git add * 
'' 
Git status

On branch develop
Your branch is up to date with 'origin/develop'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Git_Repos.ipynb

nothing added to commit but untracked files present (use "git add" to track)

The file will have its original line endings in your working directory

On branch develop
Your branch is up to date with 'origin/develop'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   Git_Repos.ipynb


In [ ]:
Git commit -m "Added a Jupyter notebook as a demo" # or some other message
''
Git log --oneline --decorate | select -First 5
#For the notebook only get the first few lines.
''
Git status

[develop 137fa6d] Added a Jupyter notebook as a demo
 1 file changed, 1217 insertions(+)
 create mode 100644 Git_Repos.ipynb

137fa6d (HEAD -> develop) Added a Jupyter notebook as a demo
5495636 (tag: 19_develop, origin/develop) Update version variables and graphs ***NO_DEPLOY***
8683936 (tag: 17_develop) Sync Release 0.1 from Master
5942680 (tag: MasterValidation_runID_18, origin/master, origin/HEAD, master) Merged PR 2: Release 0.1
5a996b2 Add Documents ***NO_CI***

On branch develop
Your branch is ahead of 'origin/develop' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean

The log shows locally we have moved forward to commit `137fa6d` but -as far as we know - the server "origin" is still on commit `5495636`. The server can't pull from us, we have to push to the server

In [ ]:
Git push

To https://dev.azure.com/CAS-DFRP/DFRP/_git/DFRP
   5495636..137fa6d  develop -> develop

It's not the clearest message ever, but it says the develop branch has moved from `5495636` to `137fa6d`    
In our case, because there is a CI pipeline which will now run, tags will be generated at the server.     
Other changes may also occur at the server. In this case I'm going to modify a file through via the Azure devops Web UI, but it could just as easily be another member of the team adding their contribution to a branch we both work on.    
Until we update what we know about the server `Git log` and `Git Status` have no idea this has happened

In [ ]:
# Changes have been made at the server, but we don't know about them 
Git log --oneline --decorate | select -First 5
#For the notebook only get the first few lines.
''
Git status

137fa6d (HEAD -> develop, origin/develop) Added a Jupyter notebook as a demo
5495636 (tag: 19_develop) Update version variables and graphs ***NO_DEPLOY***
8683936 (tag: 17_develop) Sync Release 0.1 from Master
5942680 (tag: MasterValidation_runID_18, origin/master, origin/HEAD, master) Merged PR 2: Release 0.1
5a996b2 Add Documents ***NO_CI***

On branch develop
Your branch is up to date with 'origin/develop'.

nothing to commit, working tree clean

In [ ]:
git fetch

From https://dev.azure.com/CAS-DFRP/DFRP/_git/DFRP
   137fa6d..f4843d2  develop    -> origin/develop
 * [new tag]         22_develop -> 22_develop
 * [new tag]         23_develop -> 23_develop
 * [new tag]         24_develop -> 24_develop

This tells us that new tags have been added since we last got an update and the development branch has moved forward from `137fa6d` which we have, to `f4843d2` which we don't. *Now* `Git status` can see we are one commit behind

In [ ]:
Git status

On branch develop
Your branch is behind 'origin/develop' by 1 commit, and can be fast-forwarded.
  (use "git pull" to update your local branch)

nothing to commit, working tree clean

In this case only the server version has changed since were last in sync, so git can "fast forward" us to the same checkin. As before the server can't push to us we have to pull. 

In [ ]:
Git pull

Updating 137fa6d..f4843d2
Fast-forward
 README.md | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)

In [ ]:
git log --oneline --decorate | select -first 5

f4843d2 (HEAD -> develop, tag: 24_develop, origin/develop) Updated README.md
137fa6d (tag: 23_develop) Added a Jupyter notebook as a demo
5495636 (tag: 22_develop, tag: 19_develop) Update version variables and graphs ***NO_DEPLOY***
8683936 (tag: 17_develop) Sync Release 0.1 from Master
5942680 (tag: MasterValidation_runID_18, origin/master, origin/HEAD, master) Merged PR 2: Release 0.1

Now the log shows us that the tag '24_develop' and the *develop* branch both locally and at the server, and the local HEAD all point to `f4843d2` 
It also shows the commit we made earlier, `137fa6d`, has been tagged '23_develop' and some CI process ran against the orginal server version, `5495636` while we were making that change because it has an addition tag of `22_Develop`   
The "master" branch , locally and at the server, still points to `5942680` 

-------

Let's assume this completes a peice of development work, and now we want to relase it.   
We have a "release! branch for code which is being prepared to be the next thing we deliver, while developers continue to work on the changes for the version after that in the "development" branch.    
We need to merge our changes into the "release" branch. So we start by checking out that branch. Since we don't have a copy of it yet we can check it out as before

In [ ]:
git checkout -b release origin/release
''
git log --oneline --decorate | select -first 2

Switched to a new branch 'release'
Branch 'release' set up to track remote branch 'release' from 'origin'.

9deb15a (HEAD -> release, tag: UAT, tag: SIT, tag: 16, origin/release) Sync Release 0.1 from Master
5942680 (tag: MasterValidation_runID_18, origin/master, origin/HEAD, master) Merged PR 2: Release 0.1

We can see that both branches share an ancestor: `5942680`. In this step we can see *Release* has been updated from this ancestor and the previous step *develop* has also been updated - 4 times since `5942680`. 

Because both branches have changes we can't do a fast forward, but Git *can* determine whether any changes in the two branches conflict, or if the two can be automatically merged. 

In [ ]:
git merge develop

Merge made by the 'recursive' strategy.
 .Documentation/Graphs/Branch-Develop.pipeline.svg  |   56 +
 .Documentation/Graphs/Branch-Master.pipeline.svg   |    2 +-
 .Documentation/Graphs/Branch-Release.pipeline.svg  |   42 +-
 .Documentation/Graphs/Stage.Build.pipeline.svg     |    2 +-
 .../Graphs/Stage.Deployment.pipeline.svg           |   18 +-
 .../Graphs/Steps.ConfigAzVM.pipeline.svg           |    6 +-
 .Documentation/Graphs/Steps.DeployARM.pipeline.svg |    6 +-
 CI/Branch-Develop.pipeline.yml                     |   15 +-
 CI/Branch-Master.pipeline.yml                      |    8 +-
 CI/Branch-Release.pipeline.yml                     |   16 +-
 Git_Repos.ipynb                                    | 1217 ++++++++++++++++++++
 README.md                                          |    2 +-
 12 files changed, 1331 insertions(+), 59 deletions(-)
 create mode 100644 .Documentation/Graphs/Branch-Develop.pipeline.svg
 create mode 100644 Git_Repos.ipynb

In this case two files were added - the two listed at the bottom. And the others all have a number of changes shown on the right. 

In [ ]:
git log --oneline --decorate | select -First 7

b6efae8 (HEAD -> release) Merge branch 'develop' into release
f4843d2 (tag: 24_develop, origin/develop, develop) Updated README.md
137fa6d (tag: 23_develop) Added a Jupyter notebook as a demo
5495636 (tag: 22_develop, tag: 19_develop) Update version variables and graphs ***NO_DEPLOY***
8683936 (tag: 17_develop) Sync Release 0.1 from Master
9deb15a (tag: UAT, tag: SIT, tag: 16, origin/release) Sync Release 0.1 from Master
5942680 (tag: MasterValidation_runID_18, origin/master, origin/HEAD, master) Merged PR 2: Release 0.1

We can see that we have a locally merged version of *release* which contains the commits which were made in the develop branch; and the the version of release at the origin is still the old one.
`git status` will reinforce this, and tell us to run `git push`

In [ ]:
git status
''
git push

On branch release
Your branch is ahead of 'origin/release' by 6 commits.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean

To https://dev.azure.com/CAS-DFRP/DFRP/_git/DFRP
   9deb15a..b6efae8  release -> release

# What about a new local branch
`git checkout -b` without specifying a source will create a new branch from the current one. So we can create a branch with a change in it. 

In [ ]:
git checkout -b featureX

Switched to a new branch 'featureX'

In [ ]:
echo "New Content" > featureX.txt
git add *
git commit -m "New feature 'X'"
''
git log --oneline --decorate | select -First 3

[featureX bfc8490] New feature 'X'
 1 file changed, 1 insertion(+)
 create mode 100644 featureX.txt

bfc8490 (HEAD -> featureX) New feature 'X'
5942680 (tag: MasterValidation_runID_18, origin/master, origin/HEAD, master) Merged PR 2: Release 0.1
b143567 Update Pipeline yaml files for new environment

In [ ]:
git fetch
git branch -a

  P10498824
  develop
* featureX
  master
  release
  remotes/origin/HEAD -> origin/master
  remotes/origin/P10498824
  remotes/origin/Testbranch
  remotes/origin/develop
  remotes/origin/master
  remotes/origin/release

The branch command shows the new branch (with a new commit in it) in the local repositiory, but `git push` would give an error because the local branch doesn't have a matching remote.    
Happily the error message suggests the command to fix this. And after running the command it the remote branch appears.

In [ ]:
git push --set-upstream origin featureX

Branch 'featureX' set up to track remote branch 'featureX' from 'origin'.
To https://dev.azure.com/CAS-DFRP/DFRP/_git/DFRP
 * [new branch]      featureX -> featureX

In [ ]:
git branch -a

  P10498824
  develop
* featureX
  master
  release
  remotes/origin/HEAD -> origin/master
  remotes/origin/P10498824
  remotes/origin/Testbranch
  remotes/origin/develop
  remotes/origin/featureX
  remotes/origin/master
  remotes/origin/release

The new feature will go through the same process of being merged that we have already seen, so it ends up in the current developments branch. 
Eventually featureX is just part of the product and no longer needs its own branch so we can delete it. 

In [ ]:
git checkout master
git branch -d featureX

Your branch is up to date with 'origin/master'.
Switched to branch 'master'
         'refs/remotes/origin/featureX', but not yet merged to HEAD.
Deleted branch featureX (was bfc8490).

In [ ]:
git push origin --delete featureX

To https://dev.azure.com/CAS-DFRP/DFRP/_git/DFRP
 - [deleted]         featureX

### First "Oh no I didn't mean to" example
Git has not discarded the file-blobs or the commit information which made the new feature branch, commit `bfc8490` still exists, it just doesn't have a branch label attached to it.     
We can checkout a new branch from an existing branch, or a tag name, or a commit ID. 

In [ ]:
git checkout -b GetItBack bfc8490

Switched to a new branch 'GetItBack'

In [ ]:
git log --oneline | select -first 4

bfc8490 New feature 'X'
5942680 Merged PR 2: Release 0.1
b143567 Update Pipeline yaml files for new environment
eeb16da Remove old test section from ARM template